In [1]:
import os
import shutil
import tarfile
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import torch
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Intialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [6]:
model.load_weights(F"/content/gdrive/MyDrive/Colab Notebooks/NLP_ClassProject/Model_TweetSentimentClassifier")


In [7]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset

dataset = load_dataset("sentiment140")

Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

In [9]:
#Tokenize and encode the data using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
print(model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [10]:
test = pd.DataFrame.from_dict(dataset['test'])

In [11]:
test = test.drop(columns=['user', 'date', 'query'])

In [12]:
test = test[test.sentiment != 2]

In [27]:
# Function to convert score to sentiment
def to_sentiment(rating):

    rating = int(rating)

    # Convert to class
    if rating < 2:
        return 0
    else:
        return 1


# Apply to the dataset
test['sentiment'] = test.sentiment.apply(to_sentiment)

In [28]:
print(test.head(30))

                                                 text  sentiment  \
0   @stellargirl I loooooooovvvvvveee my Kindle2. ...          1   
1   Reading my kindle2...  Love it... Lee childs i...          1   
2   Ok, first assesment of the #kindle2 ...it fuck...          1   
3   @kenburbary You'll love your Kindle2. I've had...          1   
4   @mikefish  Fair enough. But i have the Kindle2...          1   
5   @richardebaker no. it is too big. I'm quite ha...          1   
6   Fuck this economy. I hate aig and their non lo...          0   
7                       Jquery is my new best friend.          1   
8                                       Loves twitter          1   
9   how can you not love Obama? he makes jokes abo...          1   
11  @Karoli I firmly believe that Obama/Pelosi hav...          0   
12  House Correspondents dinner was last night who...          1   
13  Watchin Espn..Jus seen this new Nike Commerica...          1   
14  dear nike, stop with the flywire. that shit 

In [14]:
def text_cleaning(text):
	soup = BeautifulSoup(text, "html.parser")
	text = re.sub(r'\[[^]]*\]', '', soup.get_text())
	pattern = r"[^a-zA-Z0-9\s,']"
	text = re.sub(pattern, '', text)
	return str(text)


In [15]:
test['Cleaned_sentence'] = test['text'].apply(text_cleaning).tolist()

<ipython-input-14-3c233507cf64>:2: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



In [29]:

# Test data
#test_reviews = "[CLS] " +test_df['Cleaned_sentence'] + "[SEP]"
test_reviews = test['Cleaned_sentence']
test_targets = test['sentiment']


In [30]:
x_val, x_test, y_val, y_test = train_test_split(test_reviews,
													test_targets,
													test_size=0.5,
													stratify = test_targets)

In [ ]:
print(test_reviews)

0      stellargirl I loooooooovvvvvveee my Kindle2 No...
1      Reading my kindle2  Love it Lee childs is good...
2      Ok, first assesment of the kindle2 it fucking ...
3      kenburbary You'll love your Kindle2 I've had m...
4      mikefish  Fair enough But i have the Kindle2 a...
                             ...                        
493    Ask Programming LaTeX or InDesign submitted by...
494    On that note, I hate Word I hate Pages I hate ...
495    Ahhh back in a real text editing environment I...
496    Trouble in Iran, I see Hmm Iran Iran so far aw...
497    Reading the tweets coming out of Iran The whol...
Name: Cleaned_sentence, Length: 498, dtype: object


In [31]:
max_len= 128


X_test_encoded = tokenizer.batch_encode_plus(x_test.tolist(),
											padding=True,
											truncation=True,
											max_length = max_len,
											return_tensors='tf')

In [32]:
#Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
	[X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
	y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

6/6 [==============================] - 27s 4s/step - loss: 0.3831 - accuracy: 0.8056
Test loss: 0.3830977976322174, Test accuracy: 0.8055555820465088


In [ ]:
tf_prediction = tf.nn.softmax(scores[0], axis=0).numpy()